In [39]:
from collections import Counter
import json

In [3]:
with open('./miner1-tracing.json') as f:
    raw = json.load(f)

In [7]:
all_keys = set()
for y in raw:
    all_keys.update(y.keys())
all_keys

{'args',
 'bind_id',
 'cat',
 'dur',
 'flow_in',
 'flow_out',
 'id',
 'id2',
 'name',
 'ph',
 'pid',
 's',
 'scope',
 'tdur',
 'tid',
 'ts',
 'tts'}

In [38]:
def field_aggregator(name):
    ans = list()
    for y in raw:
        ans.append(y.get(name))
    return ans

In [96]:
# Find all JS function calls
fcalls = [x for x in raw if x['name'] == 'FunctionCall']
fcalls_by_ids = {}
for c in fcalls:
    key = '{}-{}'.format(c['pid'], c['tid'])
    fcalls_by_ids.setdefault(key, []).append(c)

for k, v in fcalls_by_ids.items():
    # See https://docs.google.com/document/d/1CvAClvFfyA5R-PhYUmn5OOQtYMH4h6I0nSsKchNAySU/preview
    # It mentioned how  Duration Events work.
    # But it never confirmed that 'FunctionCall' events do not nest when pid&tid do not change
    # This confirms that they do not nest
    print('Checking sanity of', k)
    assert('BB' not in ''.join(x['ph'] for x in v))
    assert('EE' not in ''.join(x['ph'] for x in v))
    assert v[0]['ph'] =='B'
    assert ''.join(x['ph'] for x in v[:2*(len(v)//2)]) == 'BE' * (len(v)//2)
    print(' + this thread checks out')
    if len(v) %2 != 0:
        print(' * remaining event: ', v[-1]['ph'])
print('OK')

Checking sanity of 14135-775
 + this thread checks out
Checking sanity of 14135-51459
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-81923
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-53251
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-84483
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-48387
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-76291
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-72963
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-57091
 + this thread checks out
 * remaining event:  B
OK


In [112]:
# Aggregate info about JS Function calls
callFrames = Counter()
callFunctionNames = Counter()
callURLs = Counter()

for k, v in fcalls_by_ids.items(): 
    bs = [x for i, x in enumerate(v) if i % 2 == 0]
    es = [x for i, x in enumerate(v) if i % 2 != 0]
    for b, e in zip(bs, es):
        info = b.get('args', {}).get('data', {})
        info.update(e.get('args', {}).get('data', {}))
        duration = e.get('tts') - b.get('tts')
        if not info: continue
        if type(duration) is not int: continue
        callFrames[info.get('frame')] += duration / 1e6
        callFunctionNames[info.get('functionName')] += duration / 1e6
        callURLs[info.get('url')] += duration / 1e6

In [113]:
callURLs

Counter({'': 0.036821000000000007,
         'blob:https://authedmine.com/ecf0db7f-3019-4468-9f71-6cfe50cffccb': 141.75913400000013,
         'https://authedmine.com/lib/simple-ui.min.js': 0.002773999999999999,
         'https://authedmine.com/media/miner.html?key=iPda8gvGQpMBx4aALroEhqOcYSZ9ta5C&user=CS460-testing&whitelabel=0&autostart=1&throttle=0.3&threads=&background=&text=&action=&ref=&graph=': 0.0945649999999999})

In [115]:
# ! This maybe major indicator: If 'None' frame uses too much time, it's usual.
callFrames

Counter({'4A3278310ADC1F4D8DE8FE0AD4FD8528': 0.13138600000000003,
         None: 141.75913400000013,
         'FDE5E38EF5D4E19DD0D2040B43FC16E9': 0.002773999999999999})

In [116]:
# 'Crypto', 'WASM', 'Mine' aren't good JS function names
callFunctionNames

Counter({'': 0.005993,
         'Auth.onMessage': 0.001977,
         'CryptonightWASMWrapper.onMessage': 23.839099999999988,
         'CryptonightWASMWrapper.workThrottled': 117.920034,
         'Miner.onMessage': 0.0007969999999999997,
         'MinerUI.drawGraph': 0.03438200000000002,
         'MinerUI.start': 0.001244,
         'MinerUI.updateStats': 0.05893899999999997,
         '_0x569398.(anonymous function)': 0.007018000000000001,
         '_0x569398.(anonymous function).(anonymous function)': 0.00338,
         '_0xfe497a.(anonymous function).(anonymous function)': 0.020429999999999997})